In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import regularizers
from keras.preprocessing.text import Tokenizer
import nltk
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt

In [2]:
films = pd.read_csv('C:\\Users\\48516\\Desktop\\Studia\\AIKE\\FilmsGenres\\wiki_movie_plots_deduped.csv')
films = films[['Plot', 'Genre']]
films.drop(films[films['Genre'] == 'unknown'].index, inplace = True)
films.reset_index(drop = True, inplace = True)
mostFrequentGenres = films['Genre'].value_counts()[:10]
genresList = mostFrequentGenres.index.tolist()
print(mostFrequentGenres.sort_values(ascending = False))

drama        5964
comedy       4379
horror       1167
action       1098
thriller      966
romance       923
western       865
crime         568
adventure     526
musical       467
Name: Genre, dtype: int64


In [3]:
filmsFrequent = films[films['Genre'].isin(genresList)]
filmsDrama = filmsFrequent[filmsFrequent['Genre'] == 'drama'].sample(5000, replace = True)
filmsComedy = filmsFrequent[filmsFrequent['Genre'] == 'comedy'].sample(5000, replace = True)
filmsHorror = filmsFrequent[filmsFrequent['Genre'] == 'horror'].sample(5000, replace = True)
filmsAction = filmsFrequent[filmsFrequent['Genre'] == 'action'].sample(5000, replace = True)
filmsThriller = filmsFrequent[filmsFrequent['Genre'] == 'thriller'].sample(5000, replace = True)
filmsRomance = filmsFrequent[filmsFrequent['Genre'] == 'romance'].sample(5000, replace = True)
filmsWestern = filmsFrequent[filmsFrequent['Genre'] == 'western'].sample(5000, replace = True)
filmsCrime = filmsFrequent[filmsFrequent['Genre'] == 'crime'].sample(5000, replace = True)
filmsAdventure = filmsFrequent[filmsFrequent['Genre'] == 'adventure'].sample(5000, replace = True)
filmsMusical = filmsFrequent[filmsFrequent['Genre'] == 'musical'].sample(5000, replace = True)
filmsOver = pd.concat([filmsDrama, filmsComedy, filmsHorror, filmsAction, filmsThriller, filmsRomance, filmsWestern,
                       filmsCrime, filmsAdventure, filmsMusical])
filmsOver = filmsOver.sample(frac = 1).reset_index(drop = True)
print(filmsOver['Genre'].value_counts().sort_values(ascending = False))
X = filmsOver['Plot'].values
y = filmsOver['Genre'].values

encoder = LabelEncoder()
y = encoder.fit_transform(y)
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.3)

crime        5000
horror       5000
drama        5000
musical      5000
romance      5000
western      5000
thriller     5000
comedy       5000
adventure    5000
action       5000
Name: Genre, dtype: int64


In [4]:
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(XTrain)

In [5]:
XTrain = tokenizer.texts_to_sequences(XTrain)
XTest = tokenizer.texts_to_sequences(XTest)

In [6]:
from keras.preprocessing.sequence import pad_sequences

XTrain = pad_sequences(XTrain, padding = 'post', maxlen = 500)
XTest = pad_sequences(XTest, padding = 'post', maxlen = 500)
print(XTrain.shape, XTest.shape, yTrain.shape, yTest.shape)

(35000, 500) (15000, 500) (35000,) (15000,)


In [7]:
vocabSize = len(tokenizer.word_index) + 1
vocabSize

88654

In [17]:
model1 = Sequential()
model1.add(layers.Embedding(vocabSize, 50, input_length = 500))
#model1.add(layers.SpatialDropout1D(0.2))
model1.add(layers.GRU(32, kernel_regularizer = regularizers.l2(0.01)))
# model1.add(layers.Dense(8, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
# model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(10, activation = 'softmax'))
model1.compile(optimizer = optimizers.RMSprop(lr = 0.01), loss = 'sparse_categorical_crossentropy', metrics = 'acc')
history1 = model1.fit(XTrain, yTrain, epochs = 20, validation_data = (XTest, yTest), batch_size = 100)

Epoch 1/20
350/350 [==============================] - 56s 157ms/step - loss: 2.3331 - acc: 0.1114 - val_loss: 2.4982 - val_acc: 0.1380
Epoch 2/20
350/350 [==============================] - 54s 154ms/step - loss: 2.2279 - acc: 0.1597 - val_loss: 2.0372 - val_acc: 0.2429
Epoch 3/20
350/350 [==============================] - 54s 154ms/step - loss: 2.0179 - acc: 0.2421 - val_loss: 1.9976 - val_acc: 0.2846
Epoch 4/20
350/350 [==============================] - 54s 155ms/step - loss: 1.7286 - acc: 0.3860 - val_loss: 1.7277 - val_acc: 0.4169
Epoch 5/20
350/350 [==============================] - 54s 155ms/step - loss: 1.5191 - acc: 0.4734 - val_loss: 1.3794 - val_acc: 0.5387
Epoch 6/20
350/350 [==============================] - 54s 154ms/step - loss: 1.3329 - acc: 0.5585 - val_loss: 1.3023 - val_acc: 0.5765
Epoch 7/20
350/350 [==============================] - 54s 154ms/step - loss: 1.2341 - acc: 0.5930 - val_loss: 1.2136 - val_acc: 0.6034
Epoch 8/20
350/350 [==============================] - 5

In [18]:
model2 = Sequential()
model2.add(layers.Embedding(vocabSize, 50, input_length = 500))
model2.add(layers.GRU(16, kernel_regularizer = regularizers.l2(0.01)))
model2.add(layers.Dense(10, activation = 'softmax'))
model2.compile(optimizer = optimizers.RMSprop(lr = 0.01), loss = 'sparse_categorical_crossentropy', metrics = 'acc')
history2 = model2.fit(XTrain, yTrain, epochs = 20, validation_data = (XTest, yTest), batch_size = 100)

Epoch 1/20
350/350 [==============================] - 37s 103ms/step - loss: 2.3241 - acc: 0.1149 - val_loss: 2.2644 - val_acc: 0.1257
Epoch 2/20
350/350 [==============================] - 38s 109ms/step - loss: 2.2659 - acc: 0.1340 - val_loss: 2.2564 - val_acc: 0.1486
Epoch 3/20
350/350 [==============================] - 45s 128ms/step - loss: 2.1207 - acc: 0.2018 - val_loss: 1.8979 - val_acc: 0.2931
Epoch 4/20
350/350 [==============================] - 45s 128ms/step - loss: 1.8645 - acc: 0.3038 - val_loss: 1.8189 - val_acc: 0.3617
Epoch 5/20
350/350 [==============================] - 44s 126ms/step - loss: 1.6448 - acc: 0.4325 - val_loss: 1.5468 - val_acc: 0.4705
Epoch 6/20
350/350 [==============================] - 44s 125ms/step - loss: 1.4590 - acc: 0.5068 - val_loss: 1.4900 - val_acc: 0.5023
Epoch 7/20
350/350 [==============================] - 42s 121ms/step - loss: 1.3115 - acc: 0.5616 - val_loss: 1.2671 - val_acc: 0.5863
Epoch 8/20
350/350 [==============================] - 4

In [19]:
model3 = Sequential()
model3.add(layers.Embedding(vocabSize, 50, input_length = 500))
model3.add(layers.GRU(64, kernel_regularizer = regularizers.l2(0.01)))
model3.add(layers.Dense(10, activation = 'softmax'))
model3.compile(optimizer = optimizers.RMSprop(lr = 0.01), loss = 'sparse_categorical_crossentropy', metrics = 'acc')
history3 = model3.fit(XTrain, yTrain, epochs = 20, validation_data = (XTest, yTest), batch_size = 100)

Epoch 1/20
350/350 [==============================] - 84s 237ms/step - loss: 2.3491 - acc: 0.1116 - val_loss: 2.3002 - val_acc: 0.1207
Epoch 2/20
350/350 [==============================] - 92s 264ms/step - loss: 2.2766 - acc: 0.1355 - val_loss: 2.2362 - val_acc: 0.1499
Epoch 3/20
350/350 [==============================] - 96s 275ms/step - loss: 2.2504 - acc: 0.1616 - val_loss: 2.1805 - val_acc: 0.1926
Epoch 4/20
350/350 [==============================] - 95s 272ms/step - loss: 2.0468 - acc: 0.2367 - val_loss: 2.0481 - val_acc: 0.2433
Epoch 5/20
350/350 [==============================] - 95s 272ms/step - loss: 1.9011 - acc: 0.2994 - val_loss: 1.8900 - val_acc: 0.3413
Epoch 6/20
350/350 [==============================] - 96s 275ms/step - loss: 1.6475 - acc: 0.4293 - val_loss: 1.5592 - val_acc: 0.4734
Epoch 7/20
350/350 [==============================] - 96s 276ms/step - loss: 1.4957 - acc: 0.5022 - val_loss: 1.4503 - val_acc: 0.5289
Epoch 8/20
350/350 [==============================] - 9

KeyboardInterrupt: 